<a href="https://colab.research.google.com/github/flee2-trinity/Deep-Learning-in-Sports-Result-Prediction-/blob/main/EloRanking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from google.colab import drive

drive.mount('/content/gdrive')
DATA_PATH = '/content/gdrive/My Drive/2021 Fall/Thesis I'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import datetime

def findFiles(path): 
  return glob.glob(path)

In [ ]:
import csv
filenameE = '/content/gdrive/My Drive/2021 Fall/Thesis I/NBA Data/nba_elo.csv'
with open(filenameE, newline='') as f:
      reader = csv.reader(f)
      """
      print(list(reader)[0])
      ['date', 'season', 'neutral', 'playoff', 'team1', 'team2', 'elo1_pre', 'elo2_pre', 
      'elo_prob1', 'elo_prob2', 'elo1_post', 'elo2_post', 'carm-elo1_pre', 'carm-elo2_pre', 
      'carm-elo_prob1', 'carm-elo_prob2', 'carm-elo1_post', 'carm-elo2_post', 'raptor1_pre', 
      'raptor2_pre', 'raptor_prob1', 'raptor_prob2', 'score1', 
      'score2', 'quality', 'importance', 'total_rating']
      """
      next(reader)
      rawDataE = list(reader)
print(rawDataE[0])

['1946-11-01', '1947', '0', '', 'TRH', 'NYK', '1300.0', '1300.0', '0.6400649998028851', '0.3599350001971149', '1293.2767', '1306.7233', '', '', '', '', '', '', '', '', '', '', '66', '68', '0', '', '']


In [ ]:
import datetime

def readTime(ampm):
  t = 0
  tL = ampm.replace(' ',':').split(':')
  if tL[2] == "PM" and int(tL[0]) < 12:
    t += 12
  t += int(tL[0])
  t += int(tL[1])/60
  return t

seasonBegin = datetime.datetime(2018, 10, 16)
noonTime = readTime("12:00 PM")
#not using

class aGame:
  def __init__(self, date, season, playoff, team1, team2, elo1_pre, elo2_pre, 
      elo_prob1, elo_prob2, elo1_post, elo2_post, score1, score2, result = True):
    self.season = int(season)
    self.playoff = playoff != ''
    self.team1 = team1
    self.team2 = team2
    self.elo1_pre = float(elo1_pre)
    self.elo2_pre = float(elo2_pre) 
    self.elo_prob1 = float(elo_prob1)
    self.elo_prob2 = float(elo_prob2)
    self.result = result
    if result:
      self.elo1_post = float(elo1_post)
      self.elo2_post = float(elo2_post)
      self.score1 = int(score1)
      self.score2 = int(score2)
    self.date = date
    gdL = date.split("-")
    self.GameDay = datetime.datetime(int(gdL[0]), int(gdL[1]), int(gdL[2]))
    self.carm_elo = False
    self.raptor = False
  def updateCarm(self, carm_elo1_pre, carm_elo2_pre, 
    carm_elo_prob1, carm_elo_prob2, carm_elo1_post, carm_elo2_post):
    self.carm_elo = True
    self.carm_elo1_pre = float(carm_elo1_pre)
    self.carm_elo2_pre = float(carm_elo2_pre)
    self.carm_elo_prob1 = float(carm_elo_prob1)
    self.carm_elo_prob2 = float(carm_elo_prob2)
    self.carm_elo1_post = float(carm_elo1_post) 
    self.carm_elo2_post = float(carm_elo2_post)
  def updateRaptor(self, raptor1_pre, raptor2_pre, raptor_prob1, raptor_prob2):
    self.raptor = True
    self.raptor1_pre = float(raptor1_pre)
    self.raptor2_pre = float(raptor2_pre)
    self.raptor_prob1 = float(raptor_prob1)
    self.raptor_prob2 = float(raptor_prob2)
    
def aLinetoGame(aline):
  #print(aline)
  if aline[22] == '':
    newG = aGame(aline[0], aline[1], aline[3], aline[4], aline[5], aline[6],
               aline[7], aline[8], aline[9], aline[10], aline[11], 
               aline[22], aline[23], result = False)
  else:
    newG = aGame(aline[0], aline[1], aline[3], aline[4], aline[5], aline[6],
               aline[7], aline[8], aline[9], aline[10], aline[11], 
               aline[22], aline[23])
  if aline[12] != '':
    newG.updateCarm(aline[12], aline[13], aline[14], aline[15], aline[16], aline[17])
  if aline[18] != '':
    newG.updateRaptor(aline[18], aline[19], aline[20], aline[21])
  return newG

def readLines(filename):
  with open(filename, newline='') as f:
      reader = csv.reader(f)
      next(reader)
      rawDataL = list(reader)
  #print(rawDataL)
  #'ID', 'GameDay', 'GameTime', 'Home', 'HomeID', 'Visitor', 'VisitorID', 'ScoreVis', 
  #'ScoreHome', 'GameStatus', 'Venue', 'CityState', 'Neutral', 'Division', 'Conference', 'Playoffs'
  #print("RawDataLength ",len(rawDataL))
  data = []
  print(rawDataL[0])
  for oneLine in rawDataL:
    g = aLinetoGame(oneLine)
    data.append(g)
  return data

In [ ]:
allData = readLines(filenameE)

['1946-11-01', '1947', '0', '', 'TRH', 'NYK', '1300.0', '1300.0', '0.6400649998028851', '0.3599350001971149', '1293.2767', '1306.7233', '', '', '', '', '', '', '', '', '', '', '66', '68', '0', '', '']


In [ ]:
tIDL = [set() for _ in range(2023)]
for g in allData:
  tIDL[g.season].add(g.team1)
print(len(tIDL[2020]))

30


In [ ]:
def sep2Groups(allD, season, afterN):#For Ranking
  allDataP = list(filter(lambda g:not g.playoff and g.season == season, allD))
  test_daysAfterN = list(map(lambda g: g.GameDay, allDataP))
  test_daySet = set(test_daysAfterN)
  for i in range(afterN):
    test_daySet.remove(min(test_daySet))
  print(len(test_daySet))
  test_listGroups = []
  train_listGroups = []
  for daysC in sorted(test_daySet):
    test_listGroups.append(list(filter(lambda g: g.GameDay >= daysC, allDataP)))
    train_listGroups.append(list(filter(lambda g: g.GameDay < daysC, allDataP)))
  return (train_listGroups, test_listGroups)

In [ ]:
def allTeamsComb(season):
  allComb = []
  for a in tIDL[season]:
    for b in tIDL[season]:
      if a != b:
        allComb.append((a, b))
  return allComb

In [ ]:
from collections import Counter
def getRankList(rankRecord, season):
  #print(len(rankRecord))
  rC = Counter(rankRecord).most_common()
  r = list(map(lambda x: x[0], rC))
  nRank = []
  for a in tIDL[season]:
    if a not in r:
      nRank.append(len(tIDL[season]))
    else:
      nRank.append(r.index(a)+1)
  return nRank

In [ ]:
def finalRanking(data, season):
  allRecord = []
  for g in list(filter(lambda g:not g.playoff and g.season == season, allData)):
    if g.score1 >= g.score2 :
      allRecord.append(g.team1)
    else:
      allRecord.append(g.team2)
  return getRankList(allRecord, season)


In [ ]:
def getRating(xlist, team):
  #print(team)
  tempL = list(filter(lambda x: x[0] == team, xlist))
  t = max(tempL, key = lambda x: x[2])
  return t[1]

In [ ]:
import math
from statistics import mean
import scipy.stats as stats
from scipy.stats import kendalltau
def getTauElu(trainL, scheduleL, season):
  totalR_list = []
  endRaking = finalRanking(allData, season)
  powerTestList = allTeamsComb(season)
  for date in range(len(trainL)):
    eluList = []
    for g in trainL[date]:
      eluList.append((g.team1, g.elo1_post, g.GameDay))
      eluList.append((g.team2, g.elo2_post, g.GameDay))
    #for e in eluList:
      #print(e)

    powerRankRecord = []
    for a, b in powerTestList:
      if getRating(eluList, a) + 100 >= getRating(eluList, b):
        powerRankRecord.append(a)
      else:
        powerRankRecord.append(b)
    powerRankList = getRankList(powerRankRecord, season)#Power Ranking
    #print(playOffRanking)
    powTau, _ = kendalltau(endRaking, powerRankList)
    #print(powTau)
    scheduleRecord = []
    for g in scheduleL[date]:
      if getRating(eluList, g.team1) + 100 >= getRating(eluList, g.team2):
        powerRankRecord.append(g.team1)
      else:
        powerRankRecord.append(g.team2)
    currentRecord = []
    for g in trainL[date]:
      if g.score1 >= g.score2:
        currentRecord.append(g.team1)
      else:
        currentRecord.append(g.team2)
    predictEndRanking = getRankList(currentRecord + scheduleRecord, season)
    playTau, _ = kendalltau(endRaking, predictEndRanking)
    #print(playTau)
    totalR_list.append((date, powTau, playTau, season, 'elo'))
  return totalR_list    

In [ ]:
def getTauRaptor(trainL, scheduleL, season):
  totalR_list = []
  endRaking = finalRanking(allData, season)
  powerTestList = allTeamsComb(season)
  for date in range(len(trainL)):
    eluList = []
    for g in trainL[date]:
      eluList.append((g.team1, g.elo1_post, g.GameDay))
      eluList.append((g.team2, g.elo2_post, g.GameDay))
    #for e in eluList:
      #print(e)

    powerRankRecord = []
    for a, b in powerTestList:
      if getRating(eluList, a) + 100 >= getRating(eluList, b):
        powerRankRecord.append(a)
      else:
        powerRankRecord.append(b)
    powerRankList = getRankList(powerRankRecord, season)#Power Ranking
    #print(playOffRanking)
    powTau, _ = kendalltau(endRaking, powerRankList)
    #print(powTau)
    scheduleRecord = []
    for g in scheduleL[date]:
      if getRating(eluList, g.team1) + 100 >= getRating(eluList, g.team2):
        powerRankRecord.append(g.team1)
      else:
        powerRankRecord.append(g.team2)
    currentRecord = []
    for g in trainL[date]:
      if g.score1 >= g.score2:
        currentRecord.append(g.team1)
      else:
        currentRecord.append(g.team2)
    predictEndRanking = getRankList(currentRecord + scheduleRecord, season)
    playTau, _ = kendalltau(endRaking, predictEndRanking)
    #print(playTau)
    totalR_list.append((date, powTau, playTau, season, 'elo'))    

In [ ]:
def writeInRank(newRecord):
  p1Name='/content/gdrive/My Drive/2021 Fall/Thesis I/Records/elo2_20.txt'
  p1file=open(p1Name)
  p1file_content = p1file.read()
  p1file=open(p1Name,"w")
  p1file.write(p1file_content)
  for date, pow, s, season, tp in newRecord:
    p1file.write(str(date)+" "+ str(pow) + " " + str(s) + " " + str(season) + " " + str(tp) + "\n")
  p1file.write("-\n")
  p1file=open(p1Name)
  p1file_content = p1file.read()
  #print(p1file_content)
  p1file.close()

In [ ]:
def predict(season):
  trainGs, sceduleGs = sep2Groups(allData, season, 5)
  newRecord = getTauElu(trainGs, sceduleGs, season)
  writeInRank(newRecord)

In [ ]:
for s in range(2000, 2022):
  print(s)
  predict(s)

2000
157
2001
159
2002
158
2003
159
2004
159
2005
159
2006
159
2007
158
2008
157
2009
158
2010
159
2011
159
2012
115
2013
158
2014
158
2015
157
2016
156
2017
157
2018
163
2019
163
2020
145
2021
135
